In [1]:
import pandas as pd
import time
import numpy as np
import re
import pickle
import string, unicodedata
from unidecode import unidecode

In [29]:
#fungsi fungsi ini digunakan untuk implementasi
def replace_sw(teks):
    teks_tokenize = teks.split()
    for j in range(len(teks_tokenize)):
        for k in range(len(diganti)):
            if teks_tokenize[j]==diganti[k]:
                teks_tokenize[j] = ganti[k]
        for k in range(len(hapus)):
            if teks_tokenize[j] == hapus[k]:
                teks_tokenize[j]= ''
    join = ' '.join(map(str,(teks_tokenize)))
    join = re.sub('[\s]+', ' ', join)
    return join

def removePunc(str):
    str = re.sub(r'[^\w]|_',' ',str)
    str = re.sub(r"\b\d+\b", " ", str)
    str = re.sub('[\s]+', ' ', str)
    return str
#x = removePunc(replace_sw(teks))


fo = pd.read_excel('data/karakter.xlsx', sheet_name='Sheet1')
x = fo['karakter'].tolist() #
y = fo['replace'].tolist()
def gantiKarakter(str, x=x, y=y):
    for i in range(len(x)):
        if i == 0:
            n_word = str
        n_word = n_word.replace(x[i],y[i])
    return unidecode(n_word).lower()

def normalAt(str):
    ok = gantiKarakter(str)
    n_w = []
    for i in range(len(ok)):
        if ok[i] == "@" and i !=0 and ok[i-1] !=" ":
            n_w.append(" @")
        else:
            n_w.append(ok[i])
    return "".join(n_w)

def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
       
    return str
def getJtext(text):
    words = re.findall(r'[a-z0-9@.]+', text)
    return ' '.join(words)
def preprocessing1(str):
    text = cleaning(normalAt(str))
    text = getJtext(text)
    return text

def justCek(alpha, gamma, C):
    for i in C:
        if len(C)== len(gamma) and len(gamma)==len(alpha):
            continue
        else:
            print("banyaknya data gamma c dan aplha harus sama")
        if i <= 0:
            print ("Nilai C harus lebih besar dari 0")
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
factoryStop = StopWordRemoverFactory()
stemmer = factory.create_stemmer()
stopword = factoryStop.create_stop_word_remover()



In [40]:
data = pd.read_excel('data/data_komentar_clean_1.xlsx', sheet_name = 'Sheet1')
komentar = data['komentar'].tolist()
label = data['label'].tolist()
kode = data['kode'].tolist()


data_replace  = pd.read_excel('data/Corpus_kata_replace_new.xlsx', sheet_name = 'kata_repalce')
data_hapus  = pd.read_excel('data/Corpus_kata_replace_new.xlsx', sheet_name = 'kata_hapus')
diganti = data_replace['kata'].tolist()
ganti = data_replace['ganti'].tolist()
hapus = data_hapus['kata'].tolist()

In [32]:
#proses membuat vocabularry
from sklearn.feature_extraction.text import TfidfVectorizer
#kosa_kata = set()

#seleksi fitur
#kk = TfidfVectorizer()
#kk.fit_transform(komentar)
#kosa_kata = kk.get_feature_names()
#print(len(kosa_kata))
#with open('model/feature.txt', 'w') as f:
    #for item in kosa_kata:
        #f.write("%s\n" % item)

## Proses TF-IDF

In [100]:
#membuat TF-IDF
f = open("model/feature.txt")
kosa_kata = f.read().split()

tfidf = TfidfVectorizer(vocabulary = kosa_kata)
##Membuat Matrix TF-IDF
tfidf_matrix = tfidf.fit_transform(komentar)
print(tfidf_matrix.shape)
X = tfidf_matrix.toarray()
Y = np.array(label)
y=Y

(3800, 13065)


In [8]:
y=Y

## Proses Pembagian Data Latih dan Data Uji (Cara manual)
Presentasi antara data latih dan data uji adalah 25:75, 25 persen sebagai data uji dan 75 persen sebagai data latih. 

In [10]:
bagi = 4
sx = np.vsplit(X, bagi)
sy = np.split(Y, bagi)

#latih 1
x_latih1 = np.concatenate((sx[1], sx[2], sx[3]))
y_latih1 = np.concatenate((sy[1], sy[2], sy[3]))

#uji 1
x_uji1 = sx[0]
y_uji1 = sy[0]

#latih 2
x_latih2 = np.concatenate((sx[0], sx[2], sx[3]))
y_latih2 = np.concatenate((sy[0], sy[2], sy[3]))

#uji 2
x_uji2 = sx[1]
y_uji2 = sy[1]

#latih 3
x_latih3 = np.concatenate((sx[0], sx[1], sx[3]))
y_latih3 = np.concatenate((sy[0], sy[1], sy[3]))

#uji 3
x_uji3 = sx[2]
y_uji3 = sy[2]

#latih 4
x_latih4 = np.concatenate((sx[0], sx[1], sx[2]))
y_latih4 = np.concatenate((sy[0], sy[1], sy[2]))

#uji 4
x_uji4 = sx[3]
y_uji4 = sy[3]

## Proses Pemodelan

In [55]:
#Import Library
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB


In [15]:
#menentukan C, gamma dan alpha untuk percobaan
C     = [1, 10]      #Harus titik, bukan koma
gamma = [1, 0.00001]
alpha = [1, 1]
akurasi_svm = []

#justCek(C=C, gamma=gamma, alpha = alpha)


# Jangan dijalankan

In [ ]:

data = 1   #maksimal 4
print("Data Uji "+str(data))
for i in range(len(C)):
    cSVM = svm.SVC(kernel='rbf', gamma=gamma[i], C=C[i])
    comNB = ComplementNB(alpha = alpha[1])
    
    # latih classifier
    x_latih ="x_latih"+str(data)
    y_latih = "y_latih"+str(data)
    cSVM = cSVM.fit(vars()[x_latih], vars()[y_latih])
    comNB = comNB.fit(vars()[x_latih], vars()[y_latih])
    
    x_uji = "x_uji"+str(data)
    #prediksi
    P_SVM = cSVM.predict(vars()[x_uji])
    P_CNB = comNB.predict(vars()[x_uji])
    
    y_uji = "y_uji"+str(data)
    print("gamma : ", gamma[i])
    print("C     : ", C[i])
    print("Akurasi SVM          : %.2f" % round(accuracy_score(vars()[y_uji], P_SVM),2))
    print("Akurasi C Naive Bayes: %.2f" % round(accuracy_score(vars()[y_uji], P_CNB),2))
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    akurasi_svm.append(round(accuracy_score(vars()[y_uji], P_SVM),2))
    #Lumayan lama, Mohon bersabar

Data Uji 1


# Jangan Dijalankan

In [33]:
#menjalankan seluruh data uji
#prosesnya sangat lama, saran,  jangan di-run kalau belum siap

for d in range(1,5):
    data = d
    print("Data Uji "+str(data))
    for i in range(len(C)):
        cSVM = svm.SVC(kernel='rbf', gamma=gamma[i], C=C[i])
        comNB = ComplementNB(alpha = alpha[1])
    
        # latih classifier
        x_latih ="x_latih"+str(data)
        y_latih = "y_latih"+str(data)
        cSVM = cSVM.fit(vars()[x_latih], vars()[y_latih])
        comNB = comNB.fit(vars()[x_latih], vars()[y_latih])
    
        x_uji = "x_uji"+str(data)
        #prediksi
        P_SVM = cSVM.predict(vars()[x_uji])
        P_CNB = comNB.predict(vars()[x_uji])
    
        y_uji = "y_uji"+str(data)
        print("   gamma : ", gamma[i])
        print("       C : ", C[i])
        print("   Akurasi SVM          : %.2f" % round(accuracy_score(vars()[y_uji], P_SVM),2))
        print("   Akurasi C Naive Bayes: %.2f" % round(accuracy_score(vars()[y_uji], P_CNB),2))
        print('   ++++++++++++++++++++++++++++')
#Lumayan sangat lama, Mohon bersabar

Data Uji 1
   gamma :  1
       C :  1
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.93
   ++++++++++++++++++++++++++++
   gamma :  1e-05
       C :  10
   Akurasi SVM          : 0.54
   Akurasi C Naive Bayes: 0.93
   ++++++++++++++++++++++++++++
Data Uji 2
   gamma :  1
       C :  1
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.92
   ++++++++++++++++++++++++++++
   gamma :  1e-05
       C :  10
   Akurasi SVM          : 0.51
   Akurasi C Naive Bayes: 0.92
   ++++++++++++++++++++++++++++
Data Uji 3
   gamma :  1
       C :  1
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.92
   ++++++++++++++++++++++++++++
   gamma :  1e-05
       C :  10
   Akurasi SVM          : 0.56
   Akurasi C Naive Bayes: 0.92
   ++++++++++++++++++++++++++++
Data Uji 4
   gamma :  1
       C :  1
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.90
   ++++++++++++++++++++++++++++
   gamma :  1e-05
       C :  10
   Akurasi SVM          : 0.55
   Akurasi C Naive Baye

#### Cek Akurasi 1

### Uji Validasi Cross Validations

#### jangan dijalankan

In [34]:
from sklearn.model_selection import cross_val_score
cv = 4
for d in range(1,5):
    data = d
    #print("Data Uji "+str(data))
    for i in range(len(alpha)): #Jika ada SVM
    #for i in range(len(alpha)-1): #jika hanya SNB
        cSVM = svm.SVC(kernel='rbf', gamma=gamma[i], C=C[i])
        comNB = ComplementNB(alpha = alpha[i])

        print("SVM data uji "+str(data))

        tic = time.perf_counter()
        scores = cross_val_score(cSVM, X, Y, cv=cv)
        print("   gamma : ", gamma[i])
        print("       C : ", C[i])
        print(scores)
        print("Accuracy: %0.2f" % (scores.mean()))
        toc = time.perf_counter()
        print("________________________________________________________Waktu Eksekusi : ",toc - tic, "Detik")
        print(" CNB data uji "+str(data))
        tic = time.perf_counter()
        scores = cross_val_score(comNB, X, Y, cv=cv)
        print(scores)
        print("Accuracy: %0.2f" % (scores.mean()))
        toc = time.perf_counter()
        print("_________________________________________________________Waktu Eksekusi : ",toc - tic, "Detik")

SVM data uji 1
   gamma :  1
       C :  1
[0.9106204  0.91587802 0.91043203 0.89989463]
Accuracy: 0.91
________________________________________________________Waktu Eksekusi :  1187.636867843 Detik
 CNB data uji 1
[0.93059937 0.92429022 0.92096944 0.90410959]
Accuracy: 0.92
_________________________________________________________Waktu Eksekusi :  3.5080524900004093 Detik
SVM data uji 1
   gamma :  1e-05
       C :  10
[0.53943218 0.53943218 0.53951528 0.53951528]
Accuracy: 0.54
________________________________________________________Waktu Eksekusi :  1210.7622149179997 Detik
 CNB data uji 1
[0.93059937 0.92429022 0.92096944 0.90410959]
Accuracy: 0.92
_________________________________________________________Waktu Eksekusi :  3.5350401790001342 Detik
SVM data uji 2
   gamma :  1
       C :  1
[0.9106204  0.91587802 0.91043203 0.89989463]
Accuracy: 0.91
________________________________________________________Waktu Eksekusi :  1159.2712953150012 Detik
 CNB data uji 2
[0.93059937 0.924290

In [22]:
##Menentukan Gamma, C dan alpha berapa yang akan di gunakan
#mSVM = svm.SVC(kernel='rbf', gamma=0.00001, C=10)
#mCNB = ComplementNB(alpha = 1)

## K-Fold Cross Validation 
### jalanin aja dulu

In [24]:
K = 6 # K-> jumlah pembagian data latih dan data uji
kf = KFold(n_splits=K) 


from sklearn.model_selection import KFold #import librari
akurasi_svm = []
akurasi_cnb = []
count=0

C = 1
gamma = 1
alpha = 1
cSVM = svm.SVC(kernel='rbf', gamma=gamma, C=C) #model
comNB = ComplementNB(alpha = alpha)            #model
for train_index, test_index in kf.split(X):
    #membagi data latih dan data uji
    X_train, X_test = X[train_index], X[test_index] #pembagian data latih
    y_train, y_test = y[train_index], y[test_index]
    
    #Melatih Model 
    cSVM = cSVM.fit(X_train, y_train) #SVN
    comNB = comNB.fit(X_train, y_train) #CNB
    
    P_SVM = cSVM.predict(X_test)
    P_CNB = comNB.predict(X_test)
    
    print("iterasi ke-", count)
    print("   Akurasi SVM          : %.2f" % round(accuracy_score(y_test, P_SVM),2))
    akurasi_svm.append(accuracy_score(y_test, P_SVM))
    print("   Akurasi C Naive Bayes: %.2f" % round(accuracy_score(y_test, P_CNB),2))
    akurasi_cnb.append(accuracy_score(y_test, P_CNB))
    count+=1
#Menamplkan hasil akhri
print("---------------------------------")
print("C     : ", C)
print("gamma : ", gamma)
print("alpha : ", alpha)
print("----------------------------")
print("Rata-rata Akurasi SVM: %0.2f" % (sum(akurasi_svm)/len(akurasi_svm)))
print("Rata-rata Akurasi CNB: %0.2f" % (sum(akurasi_cnb)/len(akurasi_cnb)))

iterasi ke- 0
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.92
iterasi ke- 1
   Akurasi SVM          : 0.92
   Akurasi C Naive Bayes: 0.93
iterasi ke- 2
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.92
iterasi ke- 3
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.92
iterasi ke- 4
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.91
iterasi ke- 5
   Akurasi SVM          : 0.91
   Akurasi C Naive Bayes: 0.92
---------------------------------
C     :  1
gamma :  1
alpha :  1
----------------------------
Rata-rata Akurasi SVM: 0.91
Rata-rata Akurasi CNB: 0.92


In [23]:
akurasi_svm

[0.9105263157894737, 0.9084210526315789, 0.911578947368421, 0.9052631578947369]

### classification_report 

In [ ]:
#tic = time.perf_counter()
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
#prediksi latih dan uji 1
# latih classifier
#cSVM_cr = cSVM.fit(x_latih1, y_latih1)
#comNB_cr  = comNB.fit(x_latih1, y_latih1)

#prediksi
#Y_SVM = cSVM_cr.predict(x_uji1)
#from sklearn.metrics import classification_report
#y_true = y_uji1
#y_pred = Y_SVM
#print("SVM")
#target_names = ["bukan spam", "spam"]
#print(classification_report(y_true, y_pred, target_names=target_names))
#toc = time.perf_counter()
#print("Waktu Eksekusi : ",toc - tic, "Detik")

#tic = time.perf_counter()
#Y_NB = comNB_cr.predict(x_uji1)
#y_true = y_uji1
#y_pred = Y_NB
#print("")
#print("CNB")
#target_names = ["bukan spam", "spam"]
#print(classification_report(y_true, y_pred, target_names=target_names))

#toc = time.perf_counter()
#print("Waktu Eksekusi : ",toc - tic, "Detik")

## Menyimpan model

In [56]:
cSVM = svm.SVC(kernel='rbf', gamma=1, C=1)
comNB = ComplementNB(alpha = 1)
#Melatih Model untuk data secara keseluruhan (X dan Y)
cSVM = cSVM.fit(X, Y) #SVN
comNB = comNB.fit(X, Y) #CNB

#save models
filename = 'model\cSVM.sav'
pickle.dump(cSVM, open(filename, 'wb'))
filename = 'model\comNB.sav'
pickle.dump(comNB, open(filename, 'wb'))

In [46]:
len(X)

3800

|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|


##  komentar baru

In [58]:
t_komentar = komentar

#input_komentar = "Cek IG kami Kak, solusiii naiiikkk tinggiiii sampe 175cm, bahkan sampe umuur 33th masih bisa looh, Buruaaaaannn. '"
input_komentar = input("Komentar: ")
#komentar  = [cleaning2(input_komentar)]
komentar0 = preprocessing1(input_komentar)

komentar_ = stemmer.stem(str(komentar0))
komentar_ = stopword.remove(komentar_)
t_komentar[-1]=komentar_
print("==============================================================================================================================")
print(komentar0)
print("==============================================================================================================================")

#t_clean_komentar_hapus = komentar_akhir
#komentar_siap = []
#komentar_siap.append(komentar)
#vocabulary = koso_kata
t_tfidf_matrix = tfidf.fit_transform(t_komentar)

data_X = t_tfidf_matrix.toarray()
#cek 
#x_cek = data_X[len(t_clean_komentar_hapus)-1:len(t_clean_komentar_hapus)]
x_cek = data_X[len(t_komentar)-1:len(t_komentar)]
#x_cek = data_X[-1]

# prediksi data asli
P_SVM = cSVM.predict(x_cek)
P_NB = comNB.predict(x_cek)

# print prediksi
print("Prediksi SVM (TF-IDF)           : ", P_SVM)
print("Prediksi C Naive Bayes (TF-IDF) : ", P_NB)
print(t_tfidf_matrix.shape)


Komentar: lelah
lelah
Prediksi SVM (TF-IDF)           :  ['bukan spam']
Prediksi C Naive Bayes (TF-IDF) :  ['bukan spam']
(3800, 13065)


## Prediksi komentar baru lebih dari satu komentar

In [84]:
def preprocessing_implentasi(komentar):
    n_komentar = []
    for teks in komentar:
        proses = preprocessing1(teks)
        #proses = removePunc(replace_sw(proses))
        proses = removePunc(proses)
        proses = stemmer.stem(str(proses))
        proses = stopword.remove(proses)
        n_komentar.append(proses)
    return n_komentar

In [97]:
input_ = pd.read_excel("coba prediksi.xlsx") #memasukkan nama file excel berisi komentar baru (yang akan di cek)


input_list = input_['komentar'].tolist()
f_komentar = ["" for x in range (3800)]
list_komentar = input_list
#list_komentar = ["saud adaid aji", "suka kaming", "gule china"]
#list_komentar = new_komentar
join_komentar = preprocessing_implentasi(list_komentar) + f_komentar
print("*")
kokom = join_komentar[:3800]
f = open("model/feature.txt") 
voc = f.read().split()
t_tfidf = TfidfVectorizer(vocabulary = voc)
t_tfidf_matrix = tfidf.fit_transform(kokom)
cek_komentar = t_tfidf_matrix[:len(list_komentar)]
cek_komentar = cek_komentar.toarray()

#melakukan prediksi untuk komentar baru
P_SVM = cSVM.predict(cek_komentar)
P_NB = comNB.predict(cek_komentar)


dict = {'komentar':input_list, "prediksi svm":P_SVM, "prediksi CNB": P_NB}
df = pd.DataFrame(dict, columns=['komentar',"prediksi svm","prediksi CNB"])

*


In [98]:
df

,komentar,prediksi svm,prediksi CNB
0,Assalamualaikum wr.wb..!!!*PROMO BIG SALE 2018...,spam,spam
1,yg msh kangen momen2 asian games bisa klik lin...,spam,spam
2,ok klau begitu saya coba pake panelpediavip.co...,spam,spam
3,Ғollowwww IG @Marcella.beauty_specialist gaaa...,spam,spam
4,Moga lancar n sukses y kak via besuk sore diko...,bukan spam,bukan spam
5,Semingguuu yang laluuuu akuuu cobaaa cekkk ins...,spam,spam
6,THÀNKSSSSSS ÝÀ KÀK ÚDÀHHHHHHHH KÀSÌHHHHHHHH TÀ...,spam,spam
7,MÁŚĶĔŔ ŴÁĴÁĤŃŶÁ oĶĔ ßÁŃĞĔŤ ĶÁĶ ŚĔĶÁŔÁŃĞ ŴÁĴÁĤĶ...,spam,spam
8,"ÀJÀÌBBBBBBBBB ρRÓDÚKNÝÀÀÀÀÀÀ KÀK , BÀNÝÀKKKKK ...",spam,spam
